In [2]:
import os
import numpy as np
import scanpy as sc
from scipy import sparse
import pandas as pd

In [3]:
adata = sc.read(
    "/hpc/pmc_rios/2.personal/rariese/scseq_analysis/BRO/time_course/data/Braun_dev_brain/human_dev_GRCh38-3.0.0_all_layers.h5ad"
)
adata

Only considering the two last: ['.0_all_layers', '.h5ad'].
Only considering the two last: ['.0_all_layers', '.h5ad'].


AnnData object with n_obs × n_vars = 1665937 × 33538
    obs: 'Age', 'CellClass', 'Region', 'Subdivision', 'Subregion', 'Tissue', 'TopLevelCluster', 'organism_ontology_term_id', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'assay_ontology_term_id', 'sex_ontology_term_id', 'development_stage_ontology_term_id', 'donor_id', 'suspension_type', 'dissection', 'sample_id', 'cluster_id'
    var: 'Chromosome', 'End', 'Gene', 'Start', 'Strand'
    uns: 'batch_condition', 'schema_version', 'title'
    layers: 'ambiguous', 'spliced', 'unspliced'

In [4]:
adata.obs["region_class"] = (
    adata.obs["Region"].astype(str) + "_" + adata.obs["CellClass"].astype(str)
)
adata.obs["subregion_class"] = (
    adata.obs["Subregion"].astype(str) + "_" + adata.obs["CellClass"].astype(str)
)

In [6]:
#### Do QC ####
sc.pp.filter_cells(adata, min_genes=300)

# Run some formatting
adata.var["Gene"] = adata.var["Gene"].astype(str)
adata.var = adata.var.set_index("Gene", drop=False)
adata.var.index.name = None
adata.var_names_make_unique()

In [7]:
adata.write_h5ad('/hpc/pmc_rios/2.personal/rariese/scseq_analysis/BRO/time_course/data/Braun_dev_brain/braun_2024_fetal_brain_filtered.h5ad')

In [ ]:
atlas = sc.read(
    "/hpc/pmc_rios/2.personal/rariese/scseq_analysis/BRO/time_course/data/HNOCA/hnoca_pre-release_public_subset.h5ad"
)

In [ ]:
common_genes = np.intersect1d(adata.var.index, atlas.var.index)

common_idx = np.isin(adata.var.index, common_genes)
adata_harm = adata[:, common_idx]

In [ ]:
sc.pp.normalize_total(adata_harm)
sc.pp.highly_variable_genes(adata_harm, n_top_genes=2000, batch_key="donor_id")

In [ ]:
adata_harm

AnnData object with n_obs × n_vars = 1653974 × 22198
    obs: 'CellClass', 'CellCycleFraction', 'DoubletFlag', 'DoubletScore', 'Region', 'Subdivision', 'Subregion', 'Tissue', 'TopLevelCluster', 'TotalUMIs', 'organism_ontology_term_id', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'assay_ontology_term_id', 'sex_ontology_term_id', 'development_stage_ontology_term_id', 'donor_id', 'suspension_type', 'dissection', 'total_UMIs', 'sample_id', 'cluster_id', 'region_class', 'subregion_class', 'n_genes'
    var: 'Chromosome', 'End', 'Gene', 'Start', 'Strand'
    uns: 'batch_condition', 'schema_version', 'title'

In [ ]:
sc.pp.log1p(adata_harm)
sc.pp.highly_variable_genes(adata_harm, n_top_genes=2000, batch_key="donor_id")
adata_harm.write(
    "/hpc/pmc_rios/2.personal/rariese/scseq_analysis/BRO/time_course/data/Braun_dev_brain/braun_2022_fetal_brain_v2_common.h5ad"
)
adata_harm = adata_harm[:, adata_harm.var.highly_variable].copy()
adata_harm.layers["counts"] = sparse.csr_matrix(adata_harm.layers["counts"])
adata_harm.write(
    "/hpc/pmc_rios/2.personal/rariese/scseq_analysis/BRO/time_course/data/Braun_dev_brain/braun_2022_fetal_brain_v2_common_hv2k.h5ad"
)

In [ ]:


adata_harm.write(
    "/hpc/pmc_rios/2.personal/rariese/scseq_analysis/BRO/time_course/data/Braun_dev_brain/braun_2022_fetal_brain_v2_common.h5ad"
)


#### Select highvar genes ####
adata_harm = sc.read(
    "/hpc/pmc_rios/2.personal/rariese/scseq_analysis/BRO/time_course/data/Braun_dev_brain/braun_2022_fetal_brain_v2_common.h5ad"
)

sc.pp.log1p(adata_harm)
sc.pp.highly_variable_genes(adata_harm, n_top_genes=2000, batch_key="Donor")

adata_harm.write(
    "/hpc/pmc_rios/2.personal/rariese/scseq_analysis/BRO/time_course/data/Braun_dev_brain/braun_2022_fetal_brain_v2_common.h5ad"
)

# Subset to hvg
adata_harm = adata_harm[:, adata_harm.var.highly_variable].copy()
adata_harm.layers["counts"] = sparse.csr_matrix(adata_harm.layers["counts"])
adata_harm.write(
    "/hpc/pmc_rios/2.personal/rariese/scseq_analysis/BRO/time_course/data/Braun_dev_brain/braun_2022_fetal_brain_v2_common_hv2k.h5ad"
)
